In [1]:
!pip install langchain_openai python-dotenv streamlit langchain_community langserve fastapi uvicorn sse_starlette bs4 chromadb faiss-cpu gradio pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 592.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from openai import OpenAI

import gradio as gr
import random
import time
import re

from google.colab import userdata

In [3]:
# userdata.get('OPENAIKEY')

In [4]:
client = OpenAI(api_key=userdata.get('OPENAIKEY'))

In [5]:
import logging
from transformers import pipeline

# Suppress warnings
logging.getLogger("transformers").setLevel(logging.ERROR)

def qna(context, question, max_answer_len=15):
    # Replace this with your own checkpoint
    model_checkpoint = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"
    question_answerer = pipeline("question-answering", model=model_checkpoint)
    result = question_answerer(question=question, context=context, max_answer_len=max_answer_len)
    return result['answer']

In [6]:
def generate (query):
  stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        stream=True,
    )
  gpt_response = ""
  for chunk in stream:
      if chunk.choices[0].delta.content is not None:
          gpt_response = gpt_response + chunk.choices[0].delta.content

  return gpt_response

In [7]:
def web_link(chunk_overlap,chunk_size,link=None,query=None,tag_value=None):

  if(link):
    try:
      loader = WebBaseLoader(web_path=(link,), bs_kwargs=dict(parse_only = bs4.SoupStrainer(
        # class_ = ("post-header", "post-content")
        class_ = ("mw-content-ltr mw-parser-output")
      )))

      webpagedoc = loader.load()

      text_splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)

      global documents
      documents = text_splitter.split_documents(webpagedoc)

      #vector embedding and storage
      global db
      db = "DB"

      db = Chroma.from_documents(documents, OpenAIEmbeddings(openai_api_key = userdata.get('arvind')))

      return "Link Read"

    except Exception as e:
      return str(e)

  else:
      result = "No Data"
      context = ""
      if(result=="No Data"):
        result = db.similarity_search(query)
        context = result[0].page_content
        first = context
        for i in range(len(result)):
          if first != result[i].page_content:
            context += "\n" + result[i].page_content
            break

  query = query + "\nContext:" + context
  qnabot = qna(context, query)

  response = generate(query)

  return qnabot + "\n\n" + response

In [8]:
def pdf_file(path, message,slider_value1,slider_value2):
  #implement pdf query logic here

  loader = PyPDFLoader(path)
  pdfdocs = loader.load()

  text_splitter = RecursiveCharacterTextSplitter(chunk_size = slider_value1, chunk_overlap = slider_value2)
  pdfdoc = text_splitter.split_documents(pdfdocs)

  db = Chroma.from_documents(pdfdoc[:50], OpenAIEmbeddings(openai_api_key = userdata.get('OPENAIKEY')))

  query = message

  result = db.similarity_search(query)

  context = result[0].page_content

  query = query + "\nContext:" + context
  qnabot = qna(context, query)

  response = generate(query)

  return qnabot + "\n\n" + response

In [9]:
def text_file(path, message,slider_value1,slider_value2):
  #implement text query logic here

  loader = TextLoader(path)
  text_documents = loader.load()
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = slider_value1, chunk_overlap = slider_value2)
  documents = text_splitter.split_documents(text_documents)

  db = Chroma.from_documents(documents, OpenAIEmbeddings(openai_api_key = userdata.get('OPENAIKEY')))

  query = message

  result = db.similarity_search(query)

  context = result[0].page_content

  query = query + "\nContext:" + context
  qnabot = qna(context, query)

  response = generate(query)

  return qnabot + "\n\n" + response

In [10]:
########################### GLOBAL VARIABLES ####################################
user_mode = None
path_file = None     #path to the current file uploaded
slider_value1 = 10   #default value of the slider1
slider_value2 = 10   #default value of the slider2
tag_value = "<body>"
############################## FUNCTIONS ########################################
def echo(message, history):
    #message holds the input query from the user
    #history holds the chat history in the form of a list containing lists.
    #The inner list structure: [{User text},{Bot text}]

    print(history)

    #No mode
    if(user_mode==None):
      return "Please select an option"

    #web link
    if(user_mode==3):
      url_pattern = re.compile(r'https?://\S+')

      # Search for a URL in the message text
      matchCon = url_pattern.search(message)

      if matchCon:
          url = matchCon.group(0)  # Extract the matched URL
          response = web_link(link=url,chunk_size=slider_value1,chunk_overlap=slider_value2,tag_value=tag_value)
      else:
          #function to give query to appropriate web link function and return response from it
          response = web_link(query=message,chunk_size=slider_value1,chunk_overlap=slider_value2)

      return response

    #text file mode
    if(user_mode==2):
      return text_file(path_file, message,slider_value1,slider_value2) #path to current .txt file uploaded for this mode

    #pdf file mode
    if(user_mode==1):
      return pdf_file(path_file, message,slider_value1,slider_value2)  #path to current .pdf file uploaded for this mode

#---------------------------------------------------------------------------------------------#

def process_files(argument):

    global path_file
    path_file = argument

    return gr.File(label="Upload File", visible=True,interactive=True)

#---------------------------------------------------------------------------------------------#

def change_mode(choice):
    global user_mode

    if choice == "Web Link":
        user_mode = 3
        return gr.File(visible=False), gr.Button("Submit", visible=False), gr.Button("Cancel", visible=False), gr.Textbox(label="Tag", visible=True)
    elif choice == "Text File":
        user_mode = 2
        return gr.File(label="Upload File",visible=True), gr.Button("Submit", visible=True), gr.Button("Cancel", visible=True), gr.Textbox(label="Tag", visible=False)
    else:
        user_mode = 1
        return gr.File(label="Upload File",visible=True), gr.Button("Submit", visible=True), gr.Button("Cancel", visible=True), gr.Textbox(label="Tag", visible=False)

#--------------------------------------------------------------------------------------------#

def slider_change1(value):
    global slider_value1
    slider_value1 = value

#--------------------------------------------------------------------------------------------#

def slider_change2(value):
    global slider_value2
    slider_value2 = value

#--------------------------------------------------------------------------------------------#

def tag_change(value):
    global tag_value
    tag_value = value

#--------------------------------------------------------------------------------------------#

def cancel_upload():
    return gr.File(label="Upload File", visible=True,interactive=True)

############################## INTERFACE CODE ###################################
with gr.Blocks() as demo:
    #set up radio element and file input
    with gr.Row():
      radio = gr.Radio(
          ["PDF File", "Text File", "Web Link"], label="Select Mode"
      )

      # input = gr.Interface(process_files,inputs='files',outputs=None)
      with gr.Column():
        file_input = gr.File(label="Upload File", visible=False,interactive=True)
        tag_input = gr.Textbox(label="Tag", visible=False,interactive=True)

        #set up buttons
        with gr.Row():
            submit_btn = gr.Button("Submit", visible=False)
            cancel_btn = gr.Button("Cancel", visible=False)

    with gr.Row():
      slider1 = gr.Slider(10, 1000, value=10, label="Token Size", info="Number of Tokens to tune responses.",interactive=True,step=1)
      slider2 = gr.Slider(10, 1000, value=10, label="Token Overlap", info="Tokens overlap to tune responses.",interactive=True,step=1)

    #add radio element event listener
    radio.change(fn=change_mode, inputs=radio, outputs=[file_input,submit_btn,cancel_btn,tag_input])

    # Set up submit button to process files
    submit_btn.click(fn=process_files, inputs=[file_input], outputs=[file_input])

    # Set up cancel button to clear file input
    cancel_btn.click(fn=cancel_upload, inputs=None, outputs=file_input)

    # Slider1 change interface
    slider1.change(fn=slider_change1, inputs=slider1)

    # Slider2 change interface
    slider2.change(fn=slider_change2, inputs=slider2)

    tag_input.change(fn=tag_change, inputs=tag_input)

    #set up Chat Interface
    gr.ChatInterface(
        fn=echo,
        title="Doc Bot",
    )

demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://40465b5977736870c5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[['What are the three components of  a RAG system?', "knowledge database ,retriever , and LLM\n\n retriever component is responsible for retrieving relevant texts from the knowledge database. The retriever selects a subset of texts from the database that are most likely to contain the answer to the user's question. Finally, the LLM (language model) component analyzes the retrieved texts and generates a concise and informative response to the user's question."]]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://40465b5977736870c5.gradio.live
